In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from PIL import Image
import tensorflow.python.platform
import tensorflow as tf
from sklearn.utils import class_weight

from mask_to_submission import masks_to_submission
from data_extraction import load_data_unet
from unet_pred import get_pred_img_pixelwise, make_img_overlay_pixel, get_postprocessed_unet
from unet_model import create_model_unet

from keras import backend as K
from keras.layers import Input
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint


NUM_CHANNELS = 3 
PIXEL_DEPTH = 255
NUM_LABELS = 2

TRAINING_SIZE = 2
TESTING_SIZE = 50
VALIDATION_SIZE = 1
IMG_DIMENSION = 400

BATCH_SIZE = 4 
NUM_EPOCHS = 1
MAX_AUG = 1
IMG_PATCH_SIZE = 16
NOISE_LEVEL = 0.006


'''Image paths'''
data_dir = '../data/'
train_data_filename = data_dir + 'training/images/'
train_labels_filename = data_dir + 'training/groundtruth/' 
test_data_filename = data_dir + 'test_set_images'


'''Path for storing the augmented training images'''
aug_img_path = data_dir +'training/augmented/images'
aug_gt_path = data_dir + 'training/augmented/groundtruth'


'''Path to store best weights and the submission file, and the predicted images'''
weight_path = '../weights/'
weight_filename = 'chechpoint.weights.unet.hdf5'
submission_path = '../submission.csv'
pred_test_path = '../predictions_test/'


'''Path to store results after post-processing'''
postprocess_path = '../predictions_test_pp/'
pp_submission_path = '../submission_pp.csv'


Using TensorFlow backend.


In [2]:
'''Loading data'''

x_train, y_train, x_test, x_val, y_val = load_data_unet(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE,VALIDATION_SIZE, IMG_DIMENSION,
  saltpepper=NOISE_LEVEL, augment=True, MAX_AUG=MAX_AUG, augImgDir=aug_img_path , data_dir=data_dir, groundTruthDir =aug_gt_path)

Augmenting training images...
Directory  ../data/training/augmented/images  already exists, overwritten
Directory  ../data/training/augmented/groundtruth  already exists, overwritten
Train data shape:  (4, 400, 400, 3)
Train labels shape:  (4, 400, 400, 2)
Test data shape:  (50, 608, 608, 3)
Number of samples in class 1 (background):  480767
Number of samples in class 2 (road):  159233 



In [3]:
'''Computing class weights'''

classes = np.array([0,1])
class_weights = class_weight.compute_class_weight('balanced', classes, y_train[:,:,:,0].flatten())

print('Class weights: ', class_weights) 

Class weights:  [0.66560309 2.00963368]


In [4]:
'''Loading model'''

inputs = Input((IMG_DIMENSION, IMG_DIMENSION,NUM_CHANNELS))
model = create_model_unet(inputs,n_filters=32, dropout=0.05)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 400, 400, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 400, 400, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 400, 400, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
'''Compiling model'''

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [6]:
'''Defining chechpoints to ensure the best weights are stored'''

if not os.path.isdir(weight_path):
    os.mkdir(weight_path)

callbacks = [EarlyStopping(patience=10, verbose=1),
             
             ModelCheckpoint(weight_path+weight_filename, monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=True,
                             mode='max')]

In [7]:
'''Training the model'''

model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          callbacks = callbacks,
          class_weight = class_weights
          )

Train on 4 samples, validate on 1 samples
Epoch 1/1
4/4 [==============================] - 41s 10s/step - loss: 0.9287 - acc: 0.4349 - val_loss: 0.6263 - val_acc: 0.7648

Epoch 00001: val_acc improved from -inf to 0.76479, saving model to ../weights/chechpoint.weights.unet.hdf5


In [8]:
'''Ensure we do predictions using the best weights achieved during training'''

model.load_weights(weight_path+weight_filename)

In [14]:
'''Predicting on the test images, and creating overlay and groundtruth images'''

print('Creating submission...')
if not os.path.isdir(pred_test_path):
    os.mkdir(pred_test_path)
    
list_filename = []
for i in range(1,TESTING_SIZE+1):
    if (i%np.floor(TESTING_SIZE/10) == 0):
        print(str(int(np.floor(i/np.floor(TESTING_SIZE/10))*10)), '% done')
    
    gt_pred, original_img = get_pred_img_pixelwise(test_data_filename, i, 'test', model, PIXEL_DEPTH, IMG_DIMENSION,pred_test_path)
    gt_filename = pred_test_path + "gt_pred_" + str(i) + ".png"
    list_filename.append(gt_filename)
    gt_pred.save(gt_filename)

    overlay = make_img_overlay_pixel(original_img, gt_pred, PIXEL_DEPTH)
    overlay.save(pred_test_path + "overlay_" + str(i) + ".png")

Creating submission...
10 % done
20 % done
30 % done
40 % done
50 % done
60 % done
70 % done
80 % done
90 % done
100 % done


In [10]:
'''Creating the sumbission file'''

masks_to_submission(submission_path, *list_filename)
print('Submission file created, saved at', submission_path)

Submission file created, saved at ../submission.csv


**POST-PROCESSING**

In [11]:
'''Applying post-processing to the images'''

post_processed_list = []
if not os.path.isdir(postprocess_path):
    os.mkdir(postprocess_path)


for i in range(1,TESTING_SIZE+1):
    p_img = get_postprocessed_unet(pred_test_path, i, 'test')
    filename = postprocess_path + "gt_pred_pp_" + str(i) + ".png"
    post_processed_list.append(filename)
    p_img.save(filename)
    
    pred = Image.open(filename)
    pred = pred.convert('RGB')
    imageid = "/test_%d" % i
    image_filename = test_data_filename + imageid + imageid + ".png"
    original_img = Image.open(image_filename)
    
    overlay = make_img_overlay_pixel(original_img, pred, PIXEL_DEPTH)
    overlay.save(postprocess_path + "overlay_pp_" + str(i) + ".png")



'''Saving submission file after post-processing'''

masks_to_submission(pp_submission_path, *post_processed_list)
print('Submission file after post-processing created, saved at', pp_submission_path)
print('Finished.')

Submission file after post-processing created, saved at ../submission_pp.csv
Finished.
